In [377]:
import pandas as pd     
import numpy as np  

In [378]:
import preprocessor as p
import re
#from preprocessor.api import clean, tokenize, parse

In [379]:
from textblob import TextBlob
from textblob import Word

In [380]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words

In [381]:
pd.options.display.max_colwidth=300

In [382]:
raw_data = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_text'])
raw_data.head(20)

,tweet_text
0,b'@BelindaJones68 @Hillsong @ScottMorrisonMP @nswpolice Wonder if @nswpolice know about this and would they or Fuller take action?'
1,b'@Ctl_Alt_Del @cscviews @ScottMorrisonMP China doesn\xe2\x80\x99t need to raise a sweat it just buys everything'
2,b'@Ctl_Alt_Del @ScottMorrisonMP Wants to keep up the polls e.g. leaders during wartime popularity rises \xf0\x9f\xa4\xac'
3,"b'@FBIAgentsAssoc @FBIWFO @ScottMorrisonMP(Pt10)@Citi,@eBay,@amazon,@Twitter,@bbva,@Microsoft,@Outlook,\xe2\x80\xa6 https://t.co/F8wvFFNJlM'"
4,b'@BelindaJones68 @ScottMorrisonMP @BrianCHouston @nswpolice \n@NSWHealth \nYou guys going to investigate?'
5,b'Now the rest of the world! Looking at you @ScottMorrisonMP @markrutte https://t.co/gGCTW7sa22'
6,"b""@isaacfloyd13 @Hillsong @ScottMorrisonMP @nswpolice He didn't speak in tongues at this event but it would be intere\xe2\x80\xa6 https://t.co/vUR7lNXMOp"""
7,b'@EDM17503975 @Sutts1102 @Milliganreports @poornhonest @DanielAndrewsMP @ScottMorrisonMP Why would the victims of th\xe2\x80\xa6 https://t.co/IJGT0t9wyc'
8,b'@ScottMorrisonMP @SecPompeo \xe2\x80\x9caUsTRaLiA and the US aRE gOod mAtEs\xe2\x80\x9d \n\nGet new line ya fucken nerd.'
9,"b'@KezBenn @SkyNewsAust @ScottMorrisonMP So you agree with the social restrictions on movements then? And, asking me\xe2\x80\xa6 https://t.co/fq5UuQvkMo'"


# Tweet cleaning 
### Remove URL, mention, hashtags, reserved words, emoji, smiley and number

In [383]:
raw_data['tweet_text'] = np.array([ p.clean(tweet) for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,b' Wonder if know about this and would they or Fuller take action?'
1,b' China doesn\xe2\x80\x99t need to raise a sweat it just buys everything'
2,b' Wants to keep up the polls e.g. leaders during wartime popularity rises \xf0\x9f\xa4\xac'
3,"b' (Pt10),,,,,,,\xe2\x80\xa6 '"
4,b' \n \nYou guys going to investigate?'
5,b'Now the rest of the world! Looking at you '
6,"b"" He didn't speak in tongues at this event but it would be intere\xe2\x80\xa6 """
7,b' Why would the victims of th\xe2\x80\xa6 '
8,b' \xe2\x80\x9caUsTRaLiA and the US aRE gOod mAtEs\xe2\x80\x9d \n\nGet new line ya fucken nerd.'
9,"b' So you agree with the social restrictions on movements then? And, asking me\xe2\x80\xa6 '"


### Remove punctuations

In [384]:
raw_data['tweet_text'] = np.array([ re.sub(r'(@[A-Za-z0-9]+)|([^0)-9A-Za-z \t])|(\w+:\/\/\S+)|(\'|\"|\`|…)', "", tweet) for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,b Wonder if know about this and would they or Fuller take action
1,b China doesnxe2x80x99t need to raise a sweat it just buys everything
2,b Wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,"b Pt10),,,,,,,xe2x80xa6"
4,b n nYou guys going to investigate
5,bNow the rest of the world Looking at you
6,b He didnt speak in tongues at this event but it would be interexe2x80xa6
7,b Why would the victims of thxe2x80xa6
8,b xe2x80x9caUsTRaLiA and the US aRE gOod mAtEsxe2x80x9d nnGet new line ya fucken nerd.
9,"b So you agree with the social restrictions on movements then And, asking mexe2x80xa6"


### Remove first letter 'b'

In [385]:
raw_data['tweet_text'] = np.array([ tweet[1:] for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,Wonder if know about this and would they or Fuller take action
1,China doesnxe2x80x99t need to raise a sweat it just buys everything
2,Wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,"Pt10),,,,,,,xe2x80xa6"
4,n nYou guys going to investigate
5,Now the rest of the world Looking at you
6,He didnt speak in tongues at this event but it would be interexe2x80xa6
7,Why would the victims of thxe2x80xa6
8,xe2x80x9caUsTRaLiA and the US aRE gOod mAtEsxe2x80x9d nnGet new line ya fucken nerd.
9,"So you agree with the social restrictions on movements then And, asking mexe2x80xa6"


### Lower case

In [386]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
raw_data.head(20)

,tweet_text
0,wonder if know about this and would they or fuller take action
1,china doesnxe2x80x99t need to raise a sweat it just buys everything
2,wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,"pt10),,,,,,,xe2x80xa6"
4,n nyou guys going to investigate
5,now the rest of the world looking at you
6,he didnt speak in tongues at this event but it would be interexe2x80xa6
7,why would the victims of thxe2x80xa6
8,xe2x80x9caustralia and the us are good matesxe2x80x9d nnget new line ya fucken nerd.
9,"so you agree with the social restrictions on movements then and, asking mexe2x80xa6"


### Remove stop words

In [387]:
stop_words = stopwords.words('english')

In [372]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
raw_data.head(20)

,tweet_text
0,wonder know would fuller take action
1,china doesnxe2x80x99t need raise sweat buys everything
2,wants keep polls e.g. leaders wartime popularity rises xf0x9fxa4xac
3,"pt10),,,,,,,xe2x80xa6"
4,n nyou guys going investigate
5,rest world looking
6,didnt speak tongues event would interexe2x80xa6
7,would victims thxe2x80xa6
8,xe2x80x9caustralia us good matesxe2x80x9d nnget new line ya fucken nerd.
9,"agree social restrictions movements and, asking mexe2x80xa6"


### Rare words removal

In [390]:
freq = pd.Series(' '.join(raw_data['tweet_text']).split()).value_counts()[-100:]
freq = list(freq.index)
raw_data['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
raw_data.head(20)

,tweet_text
0,wonder if know about this and would they or fuller take action
1,china doesnxe2x80x99t need to raise a sweat it just buys everything
2,wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,"pt10),,,,,,,xe2x80xa6"
4,n nyou guys going to investigate
5,now the rest of the world looking at you
6,he didnt speak in tongues at this event but it would be interexe2x80xa6
7,why would the victims of thxe2x80xa6
8,xe2x80x9caustralia and the us are good matesxe2x80x9d nnget new line ya fucken nerd.
9,"so you agree with the social restrictions on movements then and, asking mexe2x80xa6"


In [389]:
freq = pd.Series(' '.join(raw_data['tweet_text']).split()).value_counts()[-100:]
freq = list(freq.index)
freq

['petexe2x80x99',
 'quashxe2x80xa6',
 '+yrs',
 'cor',
 'duddon',
 're-test',
 'printsxe2x80xa6',
 'furniture,',
 'you,,,',
 'xf0x9fx91x8cxf0x9fx91x8c',
 'fault.nthey',
 'dipshit',
 'jsf',
 'mlo.',
 'pullingxe2x80xa6',
 'aksixe2x80xa6',
 'reduced.',
 'healing.',
 '132m',
 'own..never',
 'honolulu.',
 'recycle',
 'xf0x9fx9axa8xf0x9fx9axa8xf0x9fx9axa8nn750,000',
 'winnernbut',
 'great,bout',
 'belief,',
 'ssa',
 'nnjulian',
 'sikh...',
 'oa',
 'mullahs',
 'certificaten',
 'contest.',
 'outpost',
 'lenin',
 'it.xe2x81xa9xe2x80xa6',
 'afraid,',
 '3)',
 'widow',
 'ans',
 'killer.',
 'mathnn',
 'therennand',
 'raelene',
 'accommodations',
 'backflipping',
 'penetrate',
 ',ntelephone',
 'pubmed',
 'slimeball',
 'tense',
 'fireys',
 'nation...',
 'frontlines',
 'already.....nbetter',
 'dating-',
 'cautionary',
 'family.nnbless',
 'offxf0x9fx91x8exf0x9fx98xa0',
 'returxe2x80xa6',
 'chamberlaixe2x80xa6',
 'life/livelihood',
 'jobs...oh',
 'instructive,',
 'nnrespexe2x80xa6',
 'street.xe2x80xa6',


### Lemmatization
#### we usually prefer using lemmatization over stemming

In [391]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
raw_data.head(20)

,tweet_text
0,wonder if know about this and would they or fuller take action
1,china doesnxe2x80x99t need to raise a sweat it just buy everything
2,want to keep up the poll e.g. leader during wartime popularity rise xf0x9fxa4xac
3,"pt10),,,,,,,xe2x80xa6"
4,n nyou guy going to investigate
5,now the rest of the world looking at you
6,he didnt speak in tongue at this event but it would be interexe2x80xa6
7,why would the victim of thxe2x80xa6
8,xe2x80x9caustralia and the u are good matesxe2x80x9d nnget new line ya fucken nerd.
9,"so you agree with the social restriction on movement then and, asking mexe2x80xa6"


In [398]:
raw_data.head()

,tweet_text
0,wonder if know about this and would they or fuller take action
1,china doesnxe2x80x99t need to raise a sweat it just buy everything
2,want to keep up the poll e.g. leader during wartime popularity rise xf0x9fxa4xac
3,"pt10),,,,,,,xe2x80xa6"
4,n nyou guy going to investigate


### Import other data columns

In [392]:
data = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_length','tweet_likes','tweet_retweets'])

In [393]:
processed_df = pd.concat([raw_data, data], axis=1)

In [400]:
processed_df.head()

,tweet_text,tweet_length,tweet_likes,tweet_retweets
0,wonder if know about this and would they or fuller take action,128,0,0
1,china doesnxe2x80x99t need to raise a sweat it just buy everything,99,0,0
2,want to keep up the poll e.g. leader during wartime popularity rise xf0x9fxa4xac,103,0,0
3,"pt10),,,,,,,xe2x80xa6",126,0,0
4,n nyou guy going to investigate,102,0,0


# Write to csv

In [395]:
processed_df.to_csv('data/processed/processed_data.csv',index=False)

# Not Used

### Spelling correction

In [ ]:
raw_data['tweet_text'].apply(lambda x: str(TextBlob(x).correct()))
raw_data.head(20)

### Remove non English words
#### Not working yet

In [ ]:
raw_data1 = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_text'], nrows=10)
raw_data1.head(20)

In [ ]:
raw_data1['tweet_text'] = raw_data1['tweet_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
raw_data1.head(20)

In [ ]:
raw_data1['tweet_text'] = raw_data1['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x in words.words()))
raw_data1['tweet_text'].head(20)

### tokenize 

In [ ]:
for tweet in raw_data['tweet_text'].head(10):
    blob = TextBlob(tweet).words
    for w in blob:
        if w in words.words():
            print(w)
            

    
